# 模型树
用树来对数据建模，除了把叶节点简单地设定为常数值之外，还有一种方法是把叶节点设定为分段线性函数，这里所谓的分段线性是指模型由多个线性片段组成。如果读者仍不清楚，下面很快就会给出样例来帮助理解。考虑下图中的数据，如果使用两条直线拟合是否比使用一组常数来建模好呢？答案显而易见。可以设计两条分从0.0-0.3、从0.3-1.0的直线，于是就可以得到两个线性模型。因为数据集里的一部分数据（0.0-0.3)以某个线性模型建模，而另一部分数据（0.3-1.0)则以另一个线性模型建模，因此我们说采用了所谓的分段线性模型。
<img src="modeltree.png" style="width:400px;height:300px;">
决策树相比于其他机器学习算法的优势之一在于结果更易理解。很显然，两条直线比很多节点组成一棵大树更容易解释。模型树的可解释性是它优于回归树的特点之一。另外，模型树也具有更髙的预测准确度。

下面将利用树生成算法对数据进行切分，且每份切分数据都能很容易被线性模型所表示。该算法的关键在于误差的计算。前面巳经给出了树构建的代码，但是这里仍然需要给出每次切分时用于误差计算的代码。不知道读者是否还记得之前。creaTree()函数里有两个参数从未改变过。回归树把这两个参数固定，而此处略做修改，从而将前面的代码重用于模型树。

## 回归模型回顾
### 矩阵求导术
首先来琢磨一下定义，标量f对矩阵X的导数，定义为$\frac{\partial{f}}{\partial X}=\left [\frac{\partial{f}}{\partial X_{ij}} \right ]$,即$f$对X逐元素求导排成与X尺寸相同的矩阵。然而，这个定义在计算中并不好用，实用上的原因是在对较复杂的函数难以逐元素求导；哲理上的原因是逐元素求导破坏了整体性。试想，为何要将$f$看做矩阵$X$而不是各元素$X_{ij}$的函数呢？答案是用矩阵运算更整洁。所以在求导时不宜拆开矩阵，而是要找一个从整体出发的算法。

为此，我们来回顾，一元微积分中的导数（标量对标量的导数）与微分有联系：$df = f'(x)dx$；多元微积分中的梯度（标量对向量的导数）也与微分有联系：$df = \sum\limits_{i=1}^n \frac{\partial f}{\partial x_i}dx_i = \frac{\partial f}{\partial \boldsymbol{x}}^T d\boldsymbol{x}$ ，这里第一个等号是全微分公式，第二个等号表达了梯度与微分的联系：全微分$df$是$n\times1$梯度向量$\frac{\partial f}{\partial \boldsymbol{x}}$与$n\times1$微分向量$d\boldsymbol{x}$的内积；受此启发，我们将矩阵导数与微分建立联系：$df = \sum\limits_{i=1}^m \sum\limits_{j=1}^n \frac{\partial f}{\partial X_{ij}}dX_{ij} = \text{tr}\left(\frac{\partial f}{\partial X}^T dX\right)$ 。其中tr代表迹(trace)是方阵对角线元素之和，满足性质：对尺寸相同的矩阵A,B，$\text{tr}(A^TB) = \sum\limits_{i,j}A_{ij}B_{ij}$，即$\text{tr}(A^TB)$是矩阵A,B的内积。与梯度相似，这里第一个等号是全微分公式，第二个等号表达了矩阵导数与微分的联系：全微分$df$是$m\times n$导数$\frac{\partial f}{\partial X}$与$m\times n$微分矩阵$dX$的内积。

在运用这个特性之前，我们先来建立一些有用的法则:
1. 加减法：$d(X\pm Y) = dX \pm dY$；矩阵乘法：$d(XY) = dX Y + X dY$ ；转置：$d(X^T) = (dX)^T$；迹：$d\text{tr}(X) = \text{tr}(dX)$。
2. 逆：$dX^{-1} = -X^{-1}dX X^{-1}$。此式可在$XX^{-1}=I$两侧求微分来证明。
3. 行列式：$d|X| = \text{tr}(X^{\#}dX)$ ，其$中X^{\#}$表示X的伴随矩阵，在X可逆时又可以写作$d|X|= |X|\text{tr}(X^{-1}dX)$。
4. 逐元素乘法：$d(X\odot Y) = dX\odot Y + X\odot dY$，$\odot$表示尺寸相同的矩阵X,Y逐元素相乘。
5. 逐元素函数：$d\sigma(X) = \sigma'(X)\odot dX$ ，$\sigma(X) = \left[\sigma(X_{ij})\right]$是逐元素运算的标量函数。
6. 复合求导:设已求得$\frac{\partial f}{\partial Y}$，而Y是X的函数，如何求$\frac{\partial f}{\partial X}$呢？在微积分中有标量求导的链式法则$\frac{\partial f}{\partial x} = \frac{\partial f}{\partial y} \frac{\partial y}{\partial x}$，但这里我们不能沿用链式法则，因为矩阵对矩阵的导数$\frac{\partial Y}{\partial X}$截至目前仍是未定义的。于是我们继续追本溯源，链式法则是从何而来？源头仍然是微分。我们直接从微分入手建立复合法则：先写出$df = \text{tr}\left(\frac{\partial f}{\partial Y}^T dY\right)$，再将$dY$用$dX$表示出来代入，并使用迹技巧将其他项交换至$dX$左侧，即可得到$\frac{\partial f}{\partial X}$。

迹技巧：
1. 标量套上迹：$a = \text{tr}(a)$
2. 转置：$\mathrm{tr}(A^T) = \mathrm{tr}(A)$。
3. 线性：$\text{tr}(A\pm B) = \text{tr}(A)\pm \text{tr}(B)$
4. 矩阵乘法交换：$\text{tr}(AB) = \text{tr}(BA)$，其中$A$与$B^T$尺寸相同。两侧都等于$\sum\limits_{i,j}A_{ij}B_{ji}$
5. 矩阵乘法/逐元素乘法交换：$\text{tr}(A^T(B\odot C)) = \text{tr}((A\odot B)^TC)$，其中A, B, C尺寸相同。两侧都等于$\sum\limits_{i,j}A_{ij}B_{ij}C_{ij}$。

矩阵求导术:
<font color='red'>
若标量函数f是矩阵X经加减乘法、行列式、逆、逐元素函数等运算构成，则使用相应的运算法则对f求微分，再使用迹技巧给$df$套上迹并将其它项交换至$dX$左侧，即能得到导数。

### 线性回归的推导
损失函数$l = \|X\boldsymbol{w}- \boldsymbol{y}\|^2$， 求$\boldsymbol{w}$的最小二乘估计，即求$\frac{\partial l}{\partial \boldsymbol{w}}$的零点。其中$\boldsymbol{y}$是$m\times1$列向量，$X$是$m\times n$矩阵，$\boldsymbol{w}$是$n\times1$列向量，$l$是标量。

严格来说这是标量对向量的导数，不过可以把向量看做矩阵的特例。先将向量模平方改写成向量与自身的内积：$l = (X\boldsymbol{w}- \boldsymbol{y})^T(X\boldsymbol{w}- \boldsymbol{y})$，求微分，使用矩阵乘法、转置等法则：

先求微分:$dl = (Xd\boldsymbol{w})^T(X\boldsymbol{w}-\boldsymbol{y})+(X\boldsymbol{w}-\boldsymbol{y})^T(Xd\boldsymbol{w}) = 2(X\boldsymbol{w}-\boldsymbol{y})^TXd\boldsymbol{w}$。再对照导数与微分的联系$dl = \frac{\partial l}{\partial \boldsymbol{w}}^Td\boldsymbol{w}$，得到$\frac{\partial l}{\partial \boldsymbol{w}}= (2(X\boldsymbol{w}-\boldsymbol{y})^TX)^T = 2X^T(X\boldsymbol{w}-\boldsymbol{y})$。$\frac{\partial l}{\partial \boldsymbol{w}}$的零点即$\boldsymbol{w}$的最小二乘估计为$\boldsymbol{w} = (X^TX)^{-1}X^T\boldsymbol{y}$

In [28]:
from numpy import *
def linearSolve(dataSet):   # helper function used in two places
    m,n = shape(dataSet)
    X = mat(ones((m,n))); Y = mat(ones((m,1)))  # create a copy of data with 1 in 0th postion
    X[:,1:n] = dataSet[:,0:n-1]; Y = dataSet[:,-1]  # and strip out Y 这里X的第一列都是1，这样就可以把b写进w内
    xTx = X.T*X
    if linalg.det(xTx) == 0.0:
        raise NameError('This matrix is singular, cannot do inverse,\n\
        try increasing the second value of ops')
    ws = xTx.I * (X.T * Y)
    return ws,X,Y

In [29]:
def modelLeaf(dataSet):#create linear model and return coeficients
    ws,X,Y = linearSolve(dataSet)
    return ws

In [30]:
def modelErr(dataSet):
    ws,X,Y = linearSolve(dataSet)
    yHat = X * ws
    return sum(power(Y - yHat,2))

## 叶结点生成函数
上述程序清单中的第一个函数是linearSolve()，它会被其他两个函数调用。其主要功能是将数据集格式化成目标变量$Y$和自变量$X$，，$X$和$Y$用于执行简单的线性回归。另外在这个函数中也应当注意，如果矩阵的逆不存在也会造成程序异常。

第二个函数modelLeaf与函数regLef类似，当数据不再需要切分的时候它负责生成叶节点的模型。该函数在数据集上调用linearSolve并返回回归系数ws

最后一个函数是modelErr可以在给定的数据集上计算误差。它与regErr类似，会被比chooseBestSplit调用来找到最佳的切分。该函数在数据集上调用linearSolve ()，之后返回YHAt和Y之间的平方误差。

总的来说，createTree函数大致执行如下过程：第一次调用时，先在整个数据集上进行线性拟合，并计算其平方损失。而后遍历整个数据集寻求切分点，拟合切分好的两个数据集，并计算其平方损失，不断地进行比较，直至找到最优的切分点，接下来递归调用createTree函直至满足退出条件。

In [33]:
from tree import loadDataSet,binSplitDataSet, chooseBestSplit, createTree

In [34]:
myDat = loadDataSet('exp2.txt')
myDat = mat(myDat)
ret = createTree(myDat,modelLeaf,modelErr)
print(ret)

{'left': matrix([[1.69855694e-03],
        [1.19647739e+01]]), 'spVal': 0.285477, 'right': matrix([[3.46877936],
        [1.18521743]]), 'spInd': 0}


## 结果分析
可以看到，该代码以0.285477为界创建了两个模型，如上图所示，生成的两个线性模型分别是$y=3.4688+1.1852x$和$y=0.00169+0.1196x$

## 用树回归进行预测
treeForeCast函数用于遍历树，可以结合返回的树结构进行分析。regTreeEva和modelTreeEva函数都是对输入的数据进行格式化处理的，为了使返回值满足要求而设置。

In [35]:
def isTree(obj):
    return (type(obj).__name__=='dict')


In [36]:
def regTreeEval(model, inDat):
    return float(model)

In [37]:
def modelTreeEval(model, inDat):
    n = shape(inDat)[1]
    X = mat(ones((1,n+1)))
    X[:,1:n+1]=inDat
    return float(X*model)


In [38]:
def treeForeCast(tree, inData, modelEval=regTreeEval):
    if not isTree(tree): return modelEval(tree, inData)
    if inData[tree['spInd']] > tree['spVal']:
        if isTree(tree['left']): return treeForeCast(tree['left'], inData, modelEval)
        else: return modelEval(tree['left'], inData)
    else:
        if isTree(tree['right']): return treeForeCast(tree['right'], inData, modelEval)
        else: return modelEval(tree['right'], inData)

In [39]:
def createForeCast(tree, testData, modelEval=regTreeEval):
    m=len(testData)
    yHat = mat(zeros((m,1)))
    for i in range(m):
        yHat[i,0] = treeForeCast(tree, mat(testData[i]), modelEval)
    return yHat

In [49]:
# 回归树预测
trainMat = mat(loadDataSet('bikeSpeedVsIq_train.txt'))
testMat = mat(loadDataSet('bikeSpeedVsIq_test.txt'))
mytree = createTree(trainMat,ops=(1,20))
print(mytree)
yHat = createForeCast(mytree, testMat[:,0])
corrcoef(yHat,testMat[:,1],rowvar=0)[0,1]

{'left': {'left': {'left': 168.34161286956524, 'spVal': 20.0, 'right': 157.0484078846154, 'spInd': 0}, 'spVal': 17.0, 'right': {'left': 141.06067981481482, 'spVal': 14.0, 'right': 122.90893026923078, 'spInd': 0}, 'spInd': 0}, 'spVal': 10.0, 'right': {'left': 94.7066578125, 'spVal': 7.0, 'right': {'left': 69.02117757692308, 'spVal': 5.0, 'right': 50.94683665, 'spInd': 0}, 'spInd': 0}, 'spInd': 0}


0.964085231822215

In [50]:
#模型树预测
trainMat = mat(loadDataSet('bikeSpeedVsIq_train.txt'))
testMat = mat(loadDataSet('bikeSpeedVsIq_test.txt'))
mytree = createTree(trainMat,modelLeaf,modelErr,(1,20))
print(mytree)
yHat = createForeCast(mytree, testMat[:,0],modelTreeEval)
corrcoef(yHat,testMat[:,1],rowvar=0)[0,1]

{'left': {'left': {'left': {'left': matrix([[47.58621512],
        [ 5.51066299]]), 'spVal': 20.0, 'right': matrix([[37.54851927],
        [ 6.23298637]]), 'spInd': 0}, 'spVal': 16.0, 'right': matrix([[43.41251481],
        [ 6.37966738]]), 'spInd': 0}, 'spVal': 12.0, 'right': {'left': matrix([[-2.87684083],
        [10.20804482]]), 'spVal': 9.0, 'right': {'left': matrix([[-11.84548851],
        [ 12.12382261]]), 'spVal': 6.0, 'right': matrix([[-17.21714265],
        [ 13.72153115]]), 'spInd': 0}, 'spInd': 0}, 'spInd': 0}, 'spVal': 4.0, 'right': matrix([[ 68.87014372],
        [-11.78556471]]), 'spInd': 0}


0.9760412191380629